In [50]:
# Install dependencies (run once per environment)
%pip install -q dspy pandas python-dotenv

# dataset comes from here: https://www.kaggle.com/datasets/nikitpatel/invoice-ner-dataset?resource=download


Note: you may need to restart the kernel to use updated packages.


In [51]:
# Basic imports and environment setup
import os
import json
import dspy
import pandas as pd
from dotenv import load_dotenv

# Load API keys from .env (OPENAI_API_KEY is expected)
load_dotenv()

# Configure DSPy default LM similar to other notebooks
lm = dspy.LM("openai/gpt-5-mini", api_key=os.getenv("OPENAI_API_KEY"), temperature=1, max_tokens=16000)
dspy.configure(lm=lm)

print("DSPy configured for invoice extraction.")


DSPy configured for invoice extraction.


In [52]:
# Load dataset and preview a few rows
# Prefer absolute path next to this notebook; fallback to project root
nb_dir = os.path.dirname(os.path.abspath("__file__")) if "__file__" in globals() else os.path.abspath("./dspy")
candidates = [
    os.path.join(nb_dir, "invoice_ner_dataset.csv"),
    os.path.abspath("./dspy/invoice_ner_dataset.csv"),
    os.path.abspath("invoice_ner_dataset.csv"),
]
for p in candidates:
    if os.path.exists(p):
        csv_path = p
        break
else:
    raise FileNotFoundError("invoice_ner_dataset.csv not found in expected locations")

raw_df = pd.read_csv(csv_path)
print("Rows:", len(raw_df))
print(raw_df.head(2))

# Parse Final_Output JSON strings into dicts
raw_df["Final_Output"] = raw_df["Final_Output"].apply(lambda s: json.loads(s))

# Build small train/test splits for quick iteration
# Keep it tiny for a simple example; adjust as needed
train_df = raw_df.iloc[:30].copy()
valid_df = raw_df.iloc[30:40].copy()

def to_examples(df):
    examples = []
    for _, row in df.iterrows():
        examples.append(dspy.Example(text=row["Input"], target=row["Final_Output"]).with_inputs("text"))
    return examples

train_examples = to_examples(train_df)
valid_examples = to_examples(valid_df)

print(f"Train examples: {len(train_examples)}, Valid examples: {len(valid_examples)}")


Rows: 67
                                               Input  \
0  Cream and White Simple Minimalist Catering Ser...   
1  Beige Elegant Professional Business Invoice\n\...   

                                        Final_Output  
0  {"TOTAL_AMOUNT": "$1000", "DUE_AMOUNT": "$550"...  
1  {"INVOICE_NUMBER": "#01234", "BILLED_TO": "Est...  
Train examples: 30, Valid examples: 10


In [53]:
# Define a signature for extracting a JSON dict of invoice fields
class InvoiceExtraction(dspy.Signature):
    """
    Extract key-value invoice fields as a JSON dict from free-form invoice text.
    """
    text: str = dspy.InputField(description="Raw invoice text")
    rationale: str = dspy.OutputField(description="Brief reasoning, list detected fields")
    extracted: dict = dspy.OutputField(description="JSON dict with only UPPERCASE dataset keys and string values")

# Base module (simple Predict)
extractor = dspy.Predict(InvoiceExtraction)

print("Signature and extractor ready.")


Signature and extractor ready.


In [54]:
# Define a per-field accuracy metric: correct_fields / total_gold_fields
from dspy.evaluate import Evaluate


def normalize_dict(d: dict) -> dict:
    if d is None:
        return {}
    def norm_key(k: str) -> str:
        # Uppercase keys to align with dataset convention
        return str(k).strip().upper()
    def norm_val(v: str) -> str:
        s = str(v).strip()
        s = s.replace(",", "")
        s = s.replace("$ ", "$")
        s = " ".join(s.split())  # collapse internal whitespace
        return s
    return {norm_key(k): norm_val(v) for k, v in d.items()}


def field_accuracy_metric(example: dspy.Example, pred: dspy.Prediction, trace=None, pred_name=None, pred_trace=None) -> float:
    """
    Per-field accuracy metric with optional GEPA feedback.
    Returns a float for normal evaluation; when GEPA passes pred_name/pred_trace,
    returns dspy.Prediction(score=..., feedback=...).
    """
    gold = normalize_dict(example.target)
    got = normalize_dict(getattr(pred, "extracted", {}))
    if not gold:
        return 0.0 if pred_name is None else dspy.Prediction(score=0.0, feedback="No gold fields present.")

    total = len(gold)
    correct_keys = []
    mismatched = {}
    for k, v in gold.items():
        if k in got and got[k] == v:
            correct_keys.append(k)
        else:
            mismatched[k] = (v, got.get(k, "MISSING"))
    extra_keys = [k for k in got.keys() if k not in gold]

    score = len(correct_keys) / total

    if pred_name is None:
        return score

    # Build concise feedback for GEPA to refine prompts toward dataset schema
    lines = [
        f"Matched {len(correct_keys)}/{total} fields.",
    ]
    if mismatched:
        missed_list = ", ".join(list(mismatched.keys())[:8])
        lines.append(f"Missing/mismatched: {missed_list}.")
    if extra_keys:
        extra_list = ", ".join(extra_keys[:8])
        lines.append(f"Extra keys not in schema: {extra_list}.")

    feedback = "\n".join(lines)
    return dspy.Prediction(score=score, feedback=feedback)


# Provide the devset at construction per latest API
evaluate = Evaluate(devset=valid_examples, metric=field_accuracy_metric, ordered=True)

# Standard usage: pass the module directly
initial_score = evaluate(extractor)
print("Initial field accuracy on valid:", initial_score)


2025/09/16 14:12:13 INFO dspy.evaluate.evaluate: Average Metric: 3.228846153846154 / 10 (32.3%)


Initial field accuracy on valid: EvaluationResult(score=32.29, results=<list of 10 results>)


In [55]:
# Compare base vs optimized predictions on a sample
sample = valid_examples[0]
print("Sample comparison:")
print("\nINPUT TEXT:")
print("-" * 80)
print(sample.text[:300], "...")
print("-" * 80)

print("\nBASE MODEL PREDICTION:")
base_pred = extractor(text=sample.text)
print(f"Rationale: {base_pred.rationale}")
print("\nExtracted fields:")
for k,v in (base_pred.extracted or {}).items():
    print(f"{k:20s}: {v}")

print("\nGROUND TRUTH:")
print("\nExtracted fields:")
for k,v in sample.target.items():
    print(f"{k:20s}: {v}")


Sample comparison:

INPUT TEXT:
--------------------------------------------------------------------------------
Green Blue Pink Vintage Retro Freelance Invoice

NO. DESCRIPTION RATE QTY TOTAL
01 Graphic design for website 45 20 900.00
02 Graphic design for social content 45 8 360.00
03 Account management fee 280 1 280.00
04 Two days onsite 640 2 1280.00
SUB TOTAL $ 2820.00
TAX $ 225.60
S&H nil
TOTAL $ 3045.60 ...
--------------------------------------------------------------------------------

BASE MODEL PREDICTION:
Rationale: Detected invoice fields parsed from the text:
- Invoice title
- Invoice number ("NO. 001")
- Invoice date ("02.05.2024")
- From (biller) name, company, email, address, code
- To (client) name, company, email, address, code
- Line items (4 entries) with rate, quantity, totals
- Subtotal, Tax, Shipping (S&H), Total
- Bank / account details (Account Name, Bank, Acc)
- Payment instructions / terms ("Please pay within 60 days", cheques payable)
- Notes text

Extracte

In [56]:
# Optimize with GEPA (similar to comedian-agent)
from dspy import GEPA

optimizer = GEPA(
    metric=field_accuracy_metric,
    auto="light",
    num_threads=8,
    track_stats=True,
    use_merge=False,
    reflection_lm=lm,
)

optimized_program = optimizer.compile(
    extractor,
    trainset=train_examples,
    valset=valid_examples,
)

# Standard usage: evaluate the optimized program directly
opt_score = evaluate(optimized_program)
print("Optimized field accuracy on valid:", opt_score)


2025/09/16 14:12:13 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 420 metric calls of the program. This amounts to 10.50 full evals on the train+val set.
2025/09/16 14:12:13 INFO dspy.teleprompt.gepa.gepa: Using 10 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget.
GEPA Optimization:   0%|          | 0/420 [00:00<?, ?rollouts/s]

2025/09/16 14:12:13 INFO dspy.evaluate.evaluate: Average Metric: 3.228846153846154 / 10 (32.3%)
2025/09/16 14:12:13 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.3228846153846154
2025/09/16 14:12:13 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 0.3228846153846154


Average Metric: 1.26 / 3 (42.0%): 100%|██████████| 3/3 [00:00<00:00, 1315.93it/s]

2025/09/16 14:12:13 INFO dspy.evaluate.evaluate: Average Metric: 1.259049773755656 / 3 (42.0%)


2025/09/16 14:12:56 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Proposed new text for self: Task: Extract structured invoice key/value fields as a JSON dictionary from a blob of free‑form invoice text.

Goal: produce a single JSON object (dictionary) containing only the standardized keys defined below (omit keys that cannot be confidently extracted). Do not output any prose, explanation, or additional keys outside the schema unless explicitly requested.

Schema (allowed keys - include only when present/confidently extracted):

Top-level fields (strings unless otherwise noted)
- INVOICE_NUMBER — invoice identifier as written (prefer labels like "Invoice No", "Invoice Number", "INVOICE NO.")
- INVOICE_DATE — invoice date. If parseable, normalize to ISO 8601 date (YYYY-MM-DD). If not parseable, return the raw date string.
- DUE_DATE — due date, normalized to YYYY-MM-DD when possible, otherwise raw string.
- SERVICE_DATE — service date / invoice period (YYYY-MM-DD if parseable), otherwise

...

2025/09/16 16:48:57 INFO dspy.teleprompt.gepa.gepa: Iteration 49: Proposed new text for self: You are an extractor whose job is to convert a free-form invoice text into a single JSON object exactly following the canonical schema and rules below. Use all heuristics and domain rules in this instruction to extract fields precisely and conservatively. Always prioritize correctness over guessing. Produce only one JSON object and nothing else.

Canonical top-level schema (ONLY these keys may appear; include a key ONLY when the input provides data for it):
- INVOICE_NUMBER (string)
- DATE_OF_ISSUE (string)
- SERVICE_DATE (string)
- DUE_DATE (string)
- PAYMENT_TERMS (string)
- CURRENCY (string)
- SUBTOTAL (string)
- TAX_RATE (string)
- TAX_AMOUNT (string)
- TOTAL_AMOUNT (string)
- TOTAL_AMOUNT_DUE (string)
- AMOUNT_PAID (string)
- SELLER_NAME (string)
- SELLER_ADDRESS (string)
- SELLER_PHONE (string)
- SELLER_EMAIL (string)
- BUYER_NAME (string)
- BUYER_ADDRESS (string)
- BUYER_PHONE (string)


Average Metric: 0.97 / 3 (32.5%): 100%|██████████| 3/3 [00:57<00:00, 19.11s/it]

2025/09/16 16:51:38 INFO dspy.evaluate.evaluate: Average Metric: 0.9741902834008097 / 3 (32.5%)


2025/09/16 16:52:40 INFO dspy.teleprompt.gepa.gepa: Iteration 50: Proposed new text for self: You are an invoice-extraction assistant. You will receive a single free-form plain-text invoice (often OCR-like). Your job is to extract a fixed set of standardized fields and return exactly one JSON object (and nothing else) following the strict schema, types, and rules below.

VERY IMPORTANT — OUTPUT CONSTRAINTS
- Output exactly one JSON object and nothing else (no commentary, no explanation, no surrounding text).
- The JSON must contain every top-level key listed below. If a field cannot be found, set its value to null (except ITEMS which must be an array — use [] if no items).
- Do NOT add any top-level keys beyond the required list.
- Values must be of the required types. For numeric QTY/HOURS put numbers (integers or floats); for most fields use strings or null as specified; ITEMS must be an array of ITEM objects with exactly the keys specified.
- Always return valid JSON. No empty strin

Optimized field accuracy on valid: EvaluationResult(score=32.29, results=<list of 10 results>)


In [61]:
# Evaluate the optimized program on validation set
opt_score = evaluate(optimized_program)
print("Optimized field accuracy on validation:", opt_score)


2025/09/16 17:08:22 INFO dspy.evaluate.evaluate: Average Metric: 3.228846153846154 / 10 (32.3%)


Optimized field accuracy on validation: EvaluationResult(score=32.29, results=<list of 10 results>)


In [62]:
# Quick demo on a random validation example
sample = valid_examples[0]
print("Sample comparison:")
print("\nINPUT TEXT:")
print("-" * 80)
print(sample.text[:300], "...")
print("-" * 80)

print("\nBASE MODEL PREDICTION:")
base_pred = extractor(text=sample.text)
print(f"Rationale: {base_pred.rationale}")
print("\nExtracted fields:")
for k,v in (base_pred.extracted or {}).items():
    print(f"{k:20s}: {v}")

print("\nOPTIMIZED MODEL PREDICTION:")
opt_pred = optimized_program(text=sample.text)
print(f"Rationale: {opt_pred.rationale}")
print("\nExtracted fields:")
for k,v in (opt_pred.extracted or {}).items():
    print(f"{k:20s}: {v}")

print("\nGROUND TRUTH:")
print("\nExtracted fields:")
for k,v in sample.target.items():
    print(f"{k:20s}: {v}")


Sample comparison:

INPUT TEXT:
--------------------------------------------------------------------------------
Green Blue Pink Vintage Retro Freelance Invoice

NO. DESCRIPTION RATE QTY TOTAL
01 Graphic design for website 45 20 900.00
02 Graphic design for social content 45 8 360.00
03 Account management fee 280 1 280.00
04 Two days onsite 640 2 1280.00
SUB TOTAL $ 2820.00
TAX $ 225.60
S&H nil
TOTAL $ 3045.60 ...
--------------------------------------------------------------------------------

BASE MODEL PREDICTION:
Rationale: Detected invoice fields parsed from the text:
- Invoice title
- Invoice number ("NO. 001")
- Invoice date ("02.05.2024")
- From (biller) name, company, email, address, code
- To (client) name, company, email, address, code
- Line items (4 entries) with rate, quantity, totals
- Subtotal, Tax, Shipping (S&H), Total
- Bank / account details (Account Name, Bank, Acc)
- Payment instructions / terms ("Please pay within 60 days", cheques payable)
- Notes text

Extracte

In [63]:
# Compare predictions with ground truth
print("\nField Comparison:")
print("-" * 80)
for field in sample.target.keys():
    gt_value = sample.target[field]
    opt_value = opt_pred.extracted.get(field, "MISSING")
    match = "✓" if gt_value == opt_value else "✗"
    print(f"{field:20s}: {match}")
    if match == "✗":
        print(f"  Ground truth: {gt_value}")
        print(f"  Prediction:   {opt_value}")
print("-" * 80)



Field Comparison:
--------------------------------------------------------------------------------
INVOICE_NUMBER      : ✓
DATE                : ✗
  Ground truth: 02.05.2024
  Prediction:   MISSING
BILL_TO             : ✗
  Ground truth: Rosa Maria Aguado
  Prediction:   MISSING
COMPANY             : ✗
  Ground truth: Aldenaire & Partners
  Prediction:   MISSING
ADDRESS             : ✗
  Ground truth: 123 Anywhere St., Any City, ST 12345
  Prediction:   MISSING
EMAIL               : ✗
  Ground truth: hello@reallygreatsite.com
  Prediction:   MISSING
BANK_NAME           : ✓
BANK_ACCOUNT        : ✓
ITEM_DESCRIPTION    : ✗
  Ground truth: Two days onsite
  Prediction:   MISSING
QTY                 : ✗
  Ground truth: 2
  Prediction:   MISSING
RATE                : ✗
  Ground truth: 640
  Prediction:   MISSING
AMOUNT              : ✗
  Ground truth: 1280.00
  Prediction:   MISSING
SUBTOTAL            : ✗
  Ground truth: $ 2820.00
  Prediction:   2820.00
TAX                 : ✗
  Ground tr

In [64]:
# Show the last interaction for transparency
print("\nInspecting last prompt:")
dspy.inspect_history(n=1)


Inspecting last prompt:




[2025-09-16T17:08:31.058488]

System message:

Your input fields are:
1. `text` (str): Raw invoice text
Your output fields are:
1. `rationale` (str): Brief reasoning, list detected fields
2. `extracted` (dict): JSON dict with only UPPERCASE dataset keys and string values
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## text ## ]]
{text}

[[ ## rationale ## ]]
{rationale}

[[ ## extracted ## ]]
{extracted}        # note: the value you produce must adhere to the JSON schema: {"type": "object", "additionalProperties": true}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Extract key-value invoice fields as a JSON dict from free-form invoice text.


User message:

[[ ## text ## ]]
Green Blue Pink Vintage Retro Freelance Invoice

NO. DESCRIPTION RATE QTY TOTAL
01 Graphic design for website 45 20 900.00
02 Graphic design for social content 45 8 360.00
03 Account management fe